In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_ID_calib as abatement
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb2.gdx is still active and was not deleted.


## 1: Set up tree and read in data, akin to A1\_ID\_Calib

*Read data:*

In [2]:
inputfile = "techdata_dors_2.xlsx" # file
inputoth  = 'Othdata_dors_2.xlsx'
output = techdata_to_tree.load_techcats(pd.read_excel(directory['data'] + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))
modules = ["ID", "EOP"] # modules to use

*Set up trees:*

In [3]:
nts = {m: nesting_tree.nesting_tree(name=m) for m in modules}

#### 1.1: Input displacing tree

In [4]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"}) # E to C
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"}) # C to U
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # U to T
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX") # T to inputs X
nts["ID"].add_tree(output["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":"linear_out"}) # baseline technologies
nts["ID"].add_tree(output["ID"]["basetech_inputs"], tree_name="ID_BX")

#### 1.2: End of pipe tree

In [5]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"}) # C to U
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # T to U 
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX") # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*

In [6]:
nts['ID'].add_tree(directory['data']+'\\'+inputoth,tree_name='ID_Y',**{'sheet':'Y'})
DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars'}), 'first')

*Add new Q2P:*

In [7]:
output['ID']['Q2P'] = output['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Q2P': 'maps'}).database['Q2P'].vals)

#### 1.3: Namespaces

*Give modules local namespaces:*

In [8]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

*Add emissions data:*

In [9]:
DataBase.GPM_database.merge_dbs(nts['ID'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth, {'M': 'vars','M_sets': 'sets'}),'first')

## 2: Initialize model

*Initialize:*

In [10]:
gm = abatement.abate(nt=nts['ID'],tech=output,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': 'atest'})

*Add a couple of extra aliases, and update sets with elements:*

In [11]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)

In [12]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

In [13]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
eta_star = gm.get("eta").copy()

eta_star[:] = -3
sigma_star[gm.get("kno_ID_CU")] = 0.4
sigma_star[gm.get("kno_no_ID_BX")] = 0.5
sigma_star["Y"] = 0.2

db_star["sigma"] = sigma_star
db_star["eta"] = eta_star

In [14]:
name='v0'
gm.write_and_run(name=name, add_checkpoint=name)
gm.checkpoints['v1'] = gm.model_instances[name].ws.add_checkpoint()

gm.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = gm.checkpoints[name], options_run={'checkpoint': gm.checkpoints['v1']})

gm.model.database.merge_dbs(gm.model.database,gm.model_instances['v0'].out_db,'second')

In [15]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
sigma_star[gm.get("kno_ID_CU")] = 0.87
db_star["sigma"] = sigma_star

In [16]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(gm.model.database, db_star, n_steps = 80, loop_name='sigma_loop')

In [17]:
store_sol = {"PwThat":{'domains': shock_kwargs['loop_name']}, "qD":{'domains': shock_kwargs['loop_name']}, "sigma":{'domains': shock_kwargs['loop_name']}}

In [18]:
mi = gm.model_instances[name]

In [19]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)

In [20]:
mi.execute_shock_from_cp(shock=shock, cp=gm.checkpoints['v1'])

In [21]:
mi.out_db.series["sol_PwThat"].vals

sigma_loop     n                
sigma_loop_1   Y_K                   1.000000
               U_ID_t2_1            25.826206
               C0_EL                 1.348010
               ID_t2                25.826206
               ID_t2_K               1.000000
                                      ...    
sigma_loop_80  Y                     1.565228
               basetech_EL_inp3      1.000000
               C_EL_1                0.134801
               ID_t2_electricity     1.000000
               basetech_EL           1.482811
Name: sol_PwThat, Length: 1520, dtype: float64

In [23]:
gm.model.settings.tech_catalog = "techdata_dors_2"
nodes_to_save = {
    "techdata_dors_2":["ID_t2", "U_ID_t2_1", "C_EL_1"]
}

In [24]:
sols = {key:mi.out_db.series["sol_" + key].vals for key in store_sol.keys()}

In [31]:
sols["qD"].reset_index()

,sigma_loop,n,sol_qD
0,sigma_loop_1,Y,1.209008e+02
1,sigma_loop_1,C_EL_1,4.652260e+00
2,sigma_loop_1,basetech_EL_oil,1.565134e+01
3,sigma_loop_1,Y_electricity,3.305862e+01
4,sigma_loop_1,basetech_EL_inp3,2.928098e+01
...,...,...,...
1835,sigma_loop_80,U0_ID_C0_EL,4.091767e+01
1836,sigma_loop_80,electricity,6.233982e+01
1837,sigma_loop_80,basetech_EL_electricity,2.928123e+01
1838,sigma_loop_80,Y_oil,2.573183e+01


In [25]:
out = []
x_var = "sigma"
for var in sols.keys():
    if var == x_var:
        continue
    df = sols[var]
    if len(df.index.get_level_values("n").unique()) == 1:
        only_one_series = True
    else:
        only_one_series = False
    df = df.reset_index()
    df2 = df.pivot_table(df.columns[-1], df.columns[0], df.columns[1]).reset_index()
    if gm.model.settings.tech_catalog in nodes_to_save and not only_one_series:
        df2 = df2[["sigma_loop"] + [c for c in nodes_to_save[gm.model.settings.tech_catalog] if c in df2.columns]]
    df3 = df2.merge(sols["sigma"].reset_index().drop(columns="n"), how="inner", on="sigma_loop").drop(columns="sigma_loop")
    df3 = df3.rename(columns={"sol_sigma":"sigma"}).set_index("sigma").sort_values(by="sigma")
    if only_one_series:
        df3.rename(columns={df3.columns[0]:var}, inplace=True)
    else:
        df3.rename(columns={key:var + "_" + key for key in df3.columns}, inplace=True)
    out.append(df3)

In [34]:
df3.index

Float64Index([              0.01,               0.01,               0.01,
                            0.01,               0.01,               0.01,
                            0.01,               0.01,               0.01,
                            0.01,
              ...
              0.8462025316455697, 0.8462025316455697, 0.8521518987341772,
              0.8521518987341772, 0.8581012658227849, 0.8581012658227849,
              0.8640506329113924, 0.8640506329113924,               0.87,
                            0.87],
             dtype='float64', name='sigma', length=560)

In [27]:
df = pd.concat(out, axis=1)
df

,PwThat_ID_t2,PwThat_U_ID_t2_1,PwThat_C_EL_1,qD_ID_t2,qD_U_ID_t2_1,qD_C_EL_1
sigma,,,,,,
0.010000,25.826206,25.826206,0.134801,2.136059e-08,2.136059e-08,4.652307
0.010000,25.826206,25.826206,0.134801,4.586883e-08,4.586883e-08,4.652307
0.010000,25.826206,25.826206,0.134801,4.586883e-08,4.586883e-08,4.652307
0.010000,25.826206,25.826206,0.134801,3.936741e-08,3.936741e-08,4.652307
0.010000,25.826206,25.826206,0.134801,3.936741e-08,3.936741e-08,4.652307
...,...,...,...,...,...,...
0.858101,25.826206,25.826206,0.134801,4.015233e-10,4.015233e-10,4.652307
0.864051,25.826206,25.826206,0.134801,4.015233e-10,4.015233e-10,4.652307
0.864051,25.826206,25.826206,0.134801,4.015233e-10,4.015233e-10,4.652307


## A: Additional equations

### Current application

* Define currapp\_ID[n,nn] as the share of energy-services covered by technology goods produced by technology $nn$ to satisfy the energy-service need $n$. In other words:

$$\begin{align}
    \text{currapp_ID[n,nn]} =\text{sum(nnn\$(e2t[n,nn] and e2u[n,nnn]), qD[nnn])} / qD[n].
\end{align}$$